## Business Problem

Currently I'm staying in Singapore. There are many districts in Singapore (e.g Raffles Place, Cecil, Marina, People's Park, etc.).

The list of Singapore district can be found here: https://en.wikipedia.org/wiki/Postal_codes_in_Singapore

We as a young family really enjoy Tiong Bahru whichis the oldest housing estate in Singapore.

When you visit this quaint community, you’ll immediately notice an intriguing mix of old and new. While many older residents have lived in the shophouses and flats here for decades, the younger generation flocks here for the food, shops, and culture.

https://www.visitsingapore.com/see-do-singapore/places-to-see/tiong-bahru/

The problem:
We want to live there but renting an apartment there is very expensive. We need to look for another district which is similar to Tiong Bahru but the rental is cheaper. 

In addition, I'm a frequent traveller. Thus, I want to stay somewhere not very far from the Changi airport.

The Solution:
- Classify districts into based on Points of Interests (e.g coffee shop, barkery, etc.). 
- We may also look into the rental and distance to Airport as well


## Approach:

Below are the steps:
    
1. Get the list of Singapore Neighbourhood </br>
   - Output will be a dataframe that contains the list of neighbourhood
2. Get the longitude and latitude for each neighbourhood
   - Output will be a map that visualize all the neighbourhood in Singapore
3. Get the list of Point of Interests using FourSquare for each neighbhourhood based on the longitude and latitude from Step 2
4. Do un-supervised segmentation of neighbourhood
5. Get all the neighbourhoods that are in the same cluster with Tiong Bahru 

## 1. Get the ist of Singapore Neighbourhood

### Install the library

In [149]:
!pip install beautifulsoup4

Requirement not upgraded as not directly required: beautifulsoup4 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages


### Import the libaries

In [150]:
from bs4 import BeautifulSoup # library to handle html format
import requests # library to handle requests

### Retrieve the pages from Wikipedia

In [151]:
source = requests.get('https://en.wikipedia.org/wiki/Postal_codes_in_Singapore').text
soup = BeautifulSoup(source,'lxml')

### Get the table that contains the list of all Singapore districts/neighbourhoods:


In [152]:
tb=soup.find('tbody')

In [153]:
import pandas as pd

df = pd.DataFrame(columns=('postalDistrict','Neighbourhood'))

for row in tb.find_all('tr'):
    try:
        fields = row.find_all('td')
        postalDistrict = fields[0].text.replace('\n','')
        neighborhood_list = fields[2].text.replace('\n','')
        for i in neighborhood_list.split(','):
            df=df.append(pd.DataFrame([[postalDistrict,str(i)]],columns=('postalDistrict', 'Neighbourhood')),ignore_index=True)
    
    except Exception as e:
            yt_link = None
    

### Print out the result

In [154]:
df.head()

,postalDistrict,Neighbourhood
0,01,Raffles Place
1,01,Cecil
2,01,Marina
3,01,People's Park
4,02,Anson







## 2. Get the longitude and latitude for each neighbourhood

### Install the library

In [8]:
!pip install folium # map rendering library
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab

    100% |████████████████████████████████| 92kB 7.0MB/s eta 0:00:01
Requirement not upgraded as not directly required: requests in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: jinja2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: numpy in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: six in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: chardet<3.1.0,>=3.0.2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests->folium)
Requirement not upgraded as not directly required: idna<2.7,>=2.5 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests->folium)
Requirement not upgraded as not directly required: urllib3<1.23,>=1.21.1 in /opt/conda/envs/DSX-Py

### Import the library

In [155]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [156]:
df1= pd.DataFrame(columns=('postalDistrict','Neighbourhood','longitude','latitude'))

# for each neighbourhood
#    get the longiitude and latitude by Neighbourhood name & Nonimatim library
#    and create new data frames with the following information - postalDistrict, neighbourhood, longtitude and latitude

for index, object in df.iterrows():
    try:
        postalDistrict = object['postalDistrict']
        neighbourhood = object['Neighbourhood']
        address = neighbourhood + " ,Singapore"
        
        geolocator = Nominatim(user_agent="singapore_explorer")
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        
        #print('The geograpical coordinate of {} Singapore are {}, {}.'.format(address,latitude, longitude))
        df1=df1.append(pd.DataFrame([[postalDistrict,neighbourhood,longitude,latitude]],columns=('postalDistrict', 'Neighbourhood','longitude','latitude')),ignore_index=True)
    except Exception as e:
        print('Can not longitude and latitude of {}'.format(address))
    
    

Can not longitude and latitude of  Hong Leong Garden ,Singapore
Can not longitude and latitude of  Clementi New Town ,Singapore
Can not longitude and latitude of  Beach Road (part) ,Singapore
Can not longitude and latitude of  Woodgrove ,Singapore


### Print out the result - We should see the longitude and latitude

In [157]:
df1.head()

,postalDistrict,Neighbourhood,longitude,latitude
0,01,Raffles Place,103.851568,1.283595
1,01,Cecil,103.848094,1.279265
2,01,Marina,103.852036,1.290475
3,01,People's Park,103.844160,1.285810
4,02,Anson,103.846731,1.276088


### Store the result so that we do not have to call the services to get the Long, Lat every time

In [158]:
df1.to_csv("Singapore_Neighbourhood.csv", sep='\t')

### Create a map of Singapore using latitude and longitude values

In [159]:
# create map of New York using latitude and longitude values
map_singapore= folium.Map(location=[latitude, longitude], zoom_start=12)

#('postalDistrict','neighbourhood','longitude','latitude'

# add markers to map
for lat, lng, postalDistrict, neighborhood in zip(df1['latitude'], df1['longitude'], df1['postalDistrict'], df1['Neighbourhood']):
    label = '{}, district {}'.format(neighborhood, postalDistrict)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_singapore)  
    
map_singapore

## 3. Get the list of Point of Interests using FourSquare for each neighbhourhood based on the longitude and latitude from Step 2

### Setup the FourQuare Credentail

In [160]:
CLIENT_ID = 'BZNRGRPJZMDAF2REEYRCU0NF2G1KISBPSW14FIXV0WZNDNZB' # your Foursquare ID
CLIENT_SECRET = 'VZ2T2NYEQZ2H3YKLP2CA0ODHYBXNTPPFZTHVVMD3AXHHYRUQ' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: BZNRGRPJZMDAF2REEYRCU0NF2G1KISBPSW14FIXV0WZNDNZB
CLIENT_SECRET:VZ2T2NYEQZ2H3YKLP2CA0ODHYBXNTPPFZTHVVMD3AXHHYRUQ


### Create a function to retrieve the venues for all the neighbourhood in Singapore

In [161]:
def getNearbyVenues(names, latitudes, longitudes, radius = 500, limit = 100):
    venues_list = []

    for name, lat, long in zip(names, latitudes, longitudes):

        # Create the API request URL
        url = "https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
            CLIENT_ID,
            CLIENT_SECRET,
            VERSION,
            lat,
            long,
            radius,
            limit)

        # Make the GET request
        results = requests.get(url).json()["response"]["groups"][0]["items"]

        # Return only relevant information for each nearby venue
        venues_list.append([(name,
                             lat,
                             long,
                             v["venue"]["name"],
                             v["venue"]["location"]["lat"],
                             v["venue"]["location"]["lng"],
                             v["venue"]["categories"][0]["name"]) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ["Neighbourhood",
                             "Neighbourhood Latitude",
                             "Neighbourhood Longitude",
                             "Venue",
                             "Venue Latitude",
                             "Venue Longitude",
                             "Venue Category"]
    return(nearby_venues)

### Test the function a sample neighbourhoodfrom our dataset

In [162]:
# Call the `sample` method for extracting a random row from the dataframe
sample = df1.sample().iloc[0]
sample

postalDistrict             20
Neighbourhood      Ang Mo Kio
longitude             103.847
latitude              1.36984
Name: 52, dtype: object

In [163]:
# Retrieve nearby venues for the above sample
sample_venues = getNearbyVenues(names = [sample["Neighbourhood"]],
                                latitudes = [sample["latitude"]],
                                longitudes = [sample["longitude"]])
sample_venues.head(16)

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Ang Mo Kio,1.369842,103.846609,Old Chang Kee,1.369094,103.848389,Snack Place
1,Ang Mo Kio,1.369842,103.846609,MOS Burger,1.369170,103.847831,Burger Joint
2,Ang Mo Kio,1.369842,103.846609,Subway,1.369136,103.847612,Sandwich Place
3,Ang Mo Kio,1.369842,103.846609,Bun Master,1.369242,103.849031,Bakery
4,Ang Mo Kio,1.369842,103.846609,NTUC FairPrice,1.371507,103.847082,Supermarket
5,Ang Mo Kio,1.369842,103.846609,FairPrice Xtra,1.369279,103.848886,Supermarket
6,Ang Mo Kio,1.369842,103.846609,Kam Jia Zhuang Restaurant,1.368167,103.844118,Asian Restaurant
7,Ang Mo Kio,1.369842,103.846609,PLAYe,1.369109,103.848225,Hobby Shop
8,Ang Mo Kio,1.369842,103.846609,POSB Ang Mo Kio Central Branch,1.370969,103.847341,Bank
9,Ang Mo Kio,1.369842,103.846609,Malaysia Boleh!,1.369441,103.849066,Food Court


### Apply to all of neighbourhood in our dataset

In [164]:
singapore_venues = getNearbyVenues(names = df1["Neighbourhood"],
                                 latitudes = df1["latitude"],
                                 longitudes = df1["longitude"])

In [165]:
print("Size:", singapore_venues.shape)

singapore_venues.head()

Size: (2838, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Raffles Place,1.283595,103.851568,The Fullerton Bay Hotel,1.283878,103.853314,Hotel
1,Raffles Place,1.283595,103.851568,CITY Hot Pot Shabu shabu,1.284173,103.851585,Hotpot Restaurant
2,Raffles Place,1.283595,103.851568,Virgin Active,1.284608,103.850815,Gym / Fitness Center
3,Raffles Place,1.283595,103.851568,CULINARYON,1.284876,103.850933,Comfort Food Restaurant
4,Raffles Place,1.283595,103.851568,The Salad Shop,1.285523,103.851177,Salad Place


### Analyze each neighbourhood

In [166]:
# One-hot encoding
singapore_one_hot = pd.get_dummies(singapore_venues[["Venue Category"]], prefix = "", prefix_sep = "")

# Add borough column back to dataframe
singapore_one_hot["Neighbourhood"] = singapore_venues["Neighbourhood"] 

# Move borough column to the first column
fixed_columns = [singapore_one_hot.columns[-1]] + list(singapore_one_hot.columns[:-1])
singapore_one_hot = singapore_one_hot[fixed_columns]

singapore_one_hot.head()


,Neighbourhood,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Betting Shop,Bike Trail,Bistro,Board Shop,Bookstore,Boutique,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buddhist Temple,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Cafeteria,Café,Canal,Candy Store,Cantonese Restaurant,Cheese Shop,Chinese Breakfast Place,Chinese Restaurant,Chocolate Shop,Churrascaria,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Dongbei Restaurant,Donut Shop,Dumpling Restaurant,Duty-free Shop,Electronics Store,Event Space,Farm,Fast Food Restaurant,Filipino Restaurant,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Fujian Restaurant,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Hainan Restaurant,Halal Restaurant,Harbor / Marina,Health Food Store,High School,History Museum,Hobby Shop,Hong Kong Restaurant,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Indoor Play Area,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Light Rail Station,Lighthouse,Lingerie Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Mosque,Motel,Mountain,Movie Theater,Multiplex,Museum,Music School,Music Store,Music Venue,Nature Preserve,Neighborhood,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Optical Shop,Other Great Outdoors,Outdoor Sculpture,Paella Restaurant,Pakistani Restaurant,Park,Parking,Peking Duck Restaurant,Performing Arts Venue,Pet Store,Pharmacy,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Pub,Racetrack,Ramen Restaurant,Recreation Center,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Snack Place,Soccer Stadium,Soup Place,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Stationery Store,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Swiss Restaurant,Tailor Shop,Tapas Restaurant,Taxi Stand,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Water Park,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo Exhibit
0,Raffles Place,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

### Group rows by borough by taking the mean of the frequency of occurrence for each category

In [167]:
singapore_grouped = singapore_one_hot.groupby("Neighbourhood").mean().reset_index()
singapore_grouped.head()


,Neighbourhood,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Betting Shop,Bike Trail,Bistro,Board Shop,Bookstore,Boutique,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buddhist Temple,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Cafeteria,Café,Canal,Candy Store,Cantonese Restaurant,Cheese Shop,Chinese Breakfast Place,Chinese Restaurant,Chocolate Shop,Churrascaria,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Dongbei Restaurant,Donut Shop,Dumpling Restaurant,Duty-free Shop,Electronics Store,Event Space,Farm,Fast Food Restaurant,Filipino Restaurant,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Fujian Restaurant,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Hainan Restaurant,Halal Restaurant,Harbor / Marina,Health Food Store,High School,History Museum,Hobby Shop,Hong Kong Restaurant,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Indoor Play Area,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Light Rail Station,Lighthouse,Lingerie Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Mosque,Motel,Mountain,Movie Theater,Multiplex,Museum,Music School,Music Store,Music Venue,Nature Preserve,Neighborhood,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Optical Shop,Other Great Outdoors,Outdoor Sculpture,Paella Restaurant,Pakistani Restaurant,Park,Parking,Peking Duck Restaurant,Performing Arts Venue,Pet Store,Pharmacy,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Pub,Racetrack,Ramen Restaurant,Recreation Center,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Snack Place,Soccer Stadium,Soup Place,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Stationery Store,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Swiss Restaurant,Tailor Shop,Tapas Restaurant,Taxi Stand,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Water Park,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo Exhibit
0,Amber Road,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.023256,0.0,0.0,0.046512,0.000000,0.046512,0.0,0.0,0.0,0.0,0.0,0.0,0.023256,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.023256,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.069767,0.0,0.0,0.0,0.0,0.0,0.069767,0.0,0.0,0.0,0.023256,0.023256,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.023256,0.046512,0.023256,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.023256,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0000

Define a function to sort the venues in descending order.



In [168]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending = False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [169]:
top_venues_count = 10

indicators = ["st", "nd", "rd"]

# Create columns according to number of top venues
columns = ["Neighbourhood"]

for idx in np.arange(top_venues_count):
    try:
        columns.append("{}{} Most Common Venue".format(idx + 1, indicators[idx]))
    except:
        columns.append("{}th Most Common Venue".format(idx + 1))

# Create a new dataframe
singapore_venues_sorted = pd.DataFrame(columns = columns)
singapore_venues_sorted["Neighbourhood"] = singapore_grouped["Neighbourhood"]

for idx in np.arange(singapore_grouped.shape[0]):
    singapore_venues_sorted.iloc[idx, 1:] = return_most_common_venues(singapore_grouped.iloc[idx, :], top_venues_count)

singapore_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Amber Road,Hotel,Indian Restaurant,Café,Chinese Restaurant,Pub,Japanese Restaurant,Bakery,Dessert Shop,Bar,Noodle House
1,Ang Mo Kio,Coffee Shop,Fast Food Restaurant,Dessert Shop,Bubble Tea Shop,Food Court,Japanese Restaurant,Supermarket,Sandwich Place,Modern European Restaurant,Seafood Restaurant
2,Braddell,Seafood Restaurant,Gym / Fitness Center,Zoo Exhibit,French Restaurant,Food Truck,Food Stand,Food Court,Food & Drink Shop,Food,Flower Shop
3,Bukit Panjang,Food Court,Park,Fruit & Vegetable Store,Noodle House,Miscellaneous Shop,Market,Food Stand,Food Truck,Food & Drink Shop,Farm
4,Bukit Timah,Trail,Mountain,Rest Area,Food,Farm,Fast Food Restaurant,Filipino Restaurant,Flea Market,Flower Shop,Food & Drink Shop


## 4. Do un-supervised segmentation of neighbourhood

### Run K-means to cluster the boroughs into five clusters.



In [170]:
# Set number of clusters
k_clusters = 20

singapore_grouped_clustering = singapore_grouped.drop("Neighbourhood", 1)

# Run K-means clustering
kmeans = KMeans(n_clusters = k_clusters, random_state = 0).fit(singapore_grouped_clustering)

# Check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20]

array([ 7, 19,  6, 10,  1,  7,  9,  9, 19, 19, 17, 19, 16,  7,  9,  7,  3,
       19,  7, 14], dtype=int32)

In [171]:
# Add clustering labels
singapore_venues_sorted.insert(0, "Cluster Labels", kmeans.labels_)

singapore_venues_sorted.head()


# Merge Toronto grouped data with toronto data to add latitude and longitude for each borough
#

#singapore_merged.sample(8)

,Cluster Labels,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,7,Amber Road,Hotel,Indian Restaurant,Café,Chinese Restaurant,Pub,Japanese Restaurant,Bakery,Dessert Shop,Bar,Noodle House
1,19,Ang Mo Kio,Coffee Shop,Fast Food Restaurant,Dessert Shop,Bubble Tea Shop,Food Court,Japanese Restaurant,Supermarket,Sandwich Place,Modern European Restaurant,Seafood Restaurant
2,6,Braddell,Seafood Restaurant,Gym / Fitness Center,Zoo Exhibit,French Restaurant,Food Truck,Food Stand,Food Court,Food & Drink Shop,Food,Flower Shop
3,10,Bukit Panjang,Food Court,Park,Fruit & Vegetable Store,Noodle House,Miscellaneous Shop,Market,Food Stand,Food Truck,Food & Drink Shop,Farm
4,1,Bukit Timah,Trail,Mountain,Rest Area,Food,Farm,Fast Food Restaurant,Filipino Restaurant,Flea Market,Flower Shop,Food & Drink Shop


### Merge Toronto grouped data with toronto data to add latitude and longitude for each borough

In [172]:
singapore_merged = df1
singapore_merged = singapore_merged.join(singapore_venues_sorted.set_index("Neighbourhood"), on = "Neighbourhood")
singapore_merged.sample(10)

,postalDistrict,Neighbourhood,longitude,latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
27,11,Novena,103.843492,1.320084,9,Coffee Shop,Café,Japanese Restaurant,Hotel,Hainan Restaurant,Italian Restaurant,Bakery,Dessert Shop,Ramen Restaurant,Sporting Goods Shop
57,22,Tuas,103.648236,1.329372,5,Coffee Shop,Zoo Exhibit,Event Space,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Stand,Food Court,Food & Drink Shop
7,03,Queenstown,103.806045,1.294623,14,Chinese Restaurant,Noodle House,Pool,Stadium,Seafood Restaurant,Train Station,Steakhouse,Café,Spa,Fruit & Vegetable Store
35,14,Eunos,103.902560,1.325406,16,Coffee Shop,BBQ Joint,Diner,Martial Arts Dojo,Neighborhood,Convenience Store,Japanese Restaurant,Food Truck,Food Stand,French Restaurant
11,05,Pasir Panjang,103.775657,1.288834,18,Indian Restaurant,Food Court,Asian Restaurant,Miscellaneous Shop,History Museum,Bus Station,Dog Run,Food & Drink Shop,Food Stand,Farm
39,16,Bedok,103.930216,1.323976,7,Coffee Shop,Chinese Restaurant,Food Court,Sandwich Place,Japanese Restaurant,Sushi Restaurant,Fast Food Restaurant,Supermarket,Asian Restaurant,Frozen Yogurt Shop
12,06,High Street,103.850040,1.290180,9,Japanese Restaurant,Café,Art Gallery,Nightclub,Yoga Studio,Bar,French Restaurant,Cocktail Bar,Restaurant,Spanish Restaurant
58,23,Hillview,103.767161,1.362348,9,Café,Ice Cream Shop,Pool,Coffee Shop,Italian Restaurant,Shopping Mall,Supermarket,Beer Garden,Chinese Restaurant,Sandwich Place
28,11,Thomson,103.829524,1.377716,7,Indian Restaurant,Japanese Restaurant,Snack Place,Pharmacy,Seafood Restaurant,Italian Restaurant,Buffet,Food Stand,Food Court,Food Truck
9,04,Telok Blangah,103.809863,1.270586,19,Food Court,Hotel,Coffee Shop,Chinese Restaurant,Metro Station,Asian Restaurant,Golf Course,Bus Line,Club House,Gym


### Create a map to display the cluster

In [173]:
# Create map
map_clusters = folium.Map(location = [1.283595,103.851568], zoom_start = 12)

# Set color scheme for the clusters
x = np.arange(k_clusters)
ys = [i + x + (i * x) ** 2 for i in range(k_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# Add markers to the map
markers_colors = []
for lat, long, poi, cluster in zip(singapore_merged["latitude"], singapore_merged["longitude"], singapore_merged["Neighbourhood"], singapore_merged["Cluster Labels"]):
    label = folium.Popup(str(poi) + ", cluster " + str(cluster), parse_html = True)

    folium.CircleMarker([lat, long],
                        radius = 5.0,
                        weight = 1.5,
                        popup = label,
                        color = rainbow[cluster - 1],
                        fill = True,
                        fill_color = rainbow[cluster - 1],
                        fill_opacity = 0.5).add_to(map_clusters)
map_clusters

## 5. Get all the neighbourhoods that are in the same cluster with Novena 

Examine some clusters as example

### Cluster 1

In [174]:
singapore_merged.loc[singapore_merged["Cluster Labels"] == 0,
                   singapore_merged.columns[[1] + list(range(5, singapore_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
45,Simei,Coffee Shop,Chinese Restaurant,Bus Station,Fast Food Restaurant,Hotpot Restaurant,Noodle House,Basketball Court,Soup Place,Bakery,Café
69,Sembawang,Coffee Shop,Fast Food Restaurant,Chinese Restaurant,Taxi Stand,Supermarket,Spa,Japanese Restaurant,Sushi Restaurant,Shopping Mall,Bistro


### Cluster 5

In [175]:
singapore_merged.loc[singapore_merged["Cluster Labels"] == 4,
                   singapore_merged.columns[[1] + list(range(5, singapore_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
43,Loyang,Asian Restaurant,Zoo Exhibit,Event Space,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Stand,Food Court,Food & Drink Shop


### Cluster 10

In [176]:
singapore_merged.loc[singapore_merged["Cluster Labels"] == 9,
                   singapore_merged.columns[[1] + list(range(5, singapore_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Raffles Place,Chinese Restaurant,Café,Italian Restaurant,Gym / Fitness Center,Hotel,Cocktail Bar,Bar,Yoga Studio,Japanese Restaurant,Salad Place
1,Cecil,Café,Coffee Shop,Korean Restaurant,Japanese Restaurant,Hotel,Asian Restaurant,Wine Bar,Restaurant,Food Court,Italian Restaurant
2,Marina,Cocktail Bar,Italian Restaurant,Performing Arts Venue,Hotel,French Restaurant,Art Gallery,Lounge,Restaurant,Monument / Landmark,Concert Hall
4,Anson,Coffee Shop,Japanese Restaurant,Café,Cocktail Bar,Ramen Restaurant,Korean Restaurant,Bakery,Italian Restaurant,Salad Place,Restaurant
5,Tanjong Pagar,Coffee Shop,Japanese Restaurant,Café,Bakery,Ramen Restaurant,Korean Restaurant,Cocktail Bar,Hotel,Italian Restaurant,Spanish Restaurant
6,Bukit Merah,Café,Asian Restaurant,Coffee Shop,Chinese Restaurant,Noodle House,Food Court,Japanese Restaurant,Bookstore,Ice Cream Shop,Restaurant
12,High Street,Japanese Restaurant,Café,Art Gallery,Nightclub,Yoga Studio,Bar,French Restaurant,Cocktail Bar,Restaurant,Spanish Restaurant
13,Middle Road,Café,Japanese Restaurant,Hotel,Chinese Restaurant,Food Court,Dessert Shop,Bookstore,Bakery,Ramen Restaurant,Ice Cream Shop
14,Golden Mile,Thai Restaurant,BBQ Joint,Coffee Shop,Food Court,Vietnamese Restaurant,Park,Multiplex,Burger Joint,Steakhouse,Restaurant
18,Lavender,Coffee Shop,Hotel,Café,Restaurant,Bed & Breakfast,Hostel,BBQ Joint,Food Court,Fast Food Restaurant,Vegetarian / Vegan Restaurant


### Cluster 15

In [177]:
singapore_merged.loc[singapore_merged["Cluster Labels"] == 14,
                   singapore_merged.columns[[1] + list(range(5, singapore_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Queenstown,Chinese Restaurant,Noodle House,Pool,Stadium,Seafood Restaurant,Train Station,Steakhouse,Café,Spa,Fruit & Vegetable Store
8,Tiong Bahru,Chinese Restaurant,Coffee Shop,Food Court,Noodle House,Bakery,Café,Bar,Fried Chicken Joint,Electronics Store,Hainan Restaurant
21,River Valley,Chinese Restaurant,Noodle House,Food Court,Bus Station,Asian Restaurant,Seafood Restaurant,Spa,Thai Restaurant,Yoga Studio,Dim Sum Restaurant
34,Geylang,Chinese Restaurant,Noodle House,Vegetarian / Vegan Restaurant,Food Court,Dim Sum Restaurant,Asian Restaurant,Grocery Store,Basketball Court,Steakhouse,Hostel
37,Joo Chiat,Seafood Restaurant,Grocery Store,Asian Restaurant,Malay Restaurant,Food Court,Chinese Restaurant,Juice Bar,Gas Station,Coffee Shop,Bar


### Cluster 20

In [178]:
singapore_merged.loc[singapore_merged["Cluster Labels"] == 19,
                   singapore_merged.columns[[1] + list(range(5, singapore_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Telok Blangah,Food Court,Hotel,Coffee Shop,Chinese Restaurant,Metro Station,Asian Restaurant,Golf Course,Bus Line,Club House,Gym
30,Toa Payoh,Noodle House,Chinese Restaurant,Coffee Shop,Snack Place,Asian Restaurant,Steakhouse,Food Court,Cosmetics Shop,Thai Restaurant,Café
32,Macpherson,Food Court,Furniture / Home Store,Hobby Shop,Thai Restaurant,BBQ Joint,Metro Station,Asian Restaurant,Climbing Gym,Bus Stop,Coffee Shop
41,Eastwood,Food Court,Supermarket,Ice Cream Shop,Dessert Shop,Coffee Shop,Bus Station,Resort,Filipino Restaurant,Flea Market,Flower Shop
47,Pasir Ris,Fast Food Restaurant,Chinese Restaurant,Coffee Shop,Asian Restaurant,Thai Restaurant,Park,Diner,Resort,Steakhouse,Supermarket
49,Hougang,Bus Station,Noodle House,Food Court,Steakhouse,Sandwich Place,General Entertainment,Stadium,Chinese Restaurant,Food & Drink Shop,Fast Food Restaurant
52,Ang Mo Kio,Coffee Shop,Fast Food Restaurant,Dessert Shop,Bubble Tea Shop,Food Court,Japanese Restaurant,Supermarket,Sandwich Place,Modern European Restaurant,Seafood Restaurant
54,Clementi Park,Food Court,Asian Restaurant,Indian Restaurant,Japanese Restaurant,Coffee Shop,Dessert Shop,Fast Food Restaurant,Gym,Noodle House,Electronics Store
61,Choa Chu Kang,Fast Food Restaurant,Park,Food Court,Shopping Mall,Thai Restaurant,Playground,Coffee Shop,Sandwich Place,Asian Restaurant,Event Space
64,Kranji,Racetrack,Bus Line,Gym,Café,Dessert Shop,Food Court,Go Kart Track,Lighthouse,Bus Stop,Noodle House


### Now get the cluster that  Tiong Bahru belong to

In [179]:
singapore_merged.loc[singapore_merged["Neighbourhood"] == " Tiong Bahru",
                   singapore_merged.columns[[1] + [4] + list(range(5, singapore_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Tiong Bahru,14,Chinese Restaurant,Coffee Shop,Food Court,Noodle House,Bakery,Café,Bar,Fried Chicken Joint,Electronics Store,Hainan Restaurant


As you can see Novena is in Cluster 15 (index = 14)

Let's narrow down to top 5 Most common venue

In [181]:
singapore_merged.loc[singapore_merged["Cluster Labels"] == 14,
                   singapore_merged.columns[[1] + list(range(5, 10))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
7,Queenstown,Chinese Restaurant,Noodle House,Pool,Stadium,Seafood Restaurant
8,Tiong Bahru,Chinese Restaurant,Coffee Shop,Food Court,Noodle House,Bakery
21,River Valley,Chinese Restaurant,Noodle House,Food Court,Bus Station,Asian Restaurant
34,Geylang,Chinese Restaurant,Noodle House,Vegetarian / Vegan Restaurant,Food Court,Dim Sum Restaurant
37,Joo Chiat,Seafood Restaurant,Grocery Store,Asian Restaurant,Malay Restaurant,Food Court


## Conclusion

By apply simple clustering, below neighbourhoods are similar to Tiong Bahru:
1. Queentown
2. River Valley
3. Toa Payoh
4. Geylang
5. Joo Chiat

With the help of google I can see that Geylang will be a good choce as:
1. It offers cheap rental (vs other neighhood)
2. Short distance to Changi Airport

## Thus, Geylang is the answer for my bussiness problem